In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import sys

In [131]:
# From: https://gist.github.com/vladignatyev/06860ec2040cb497f0f3
def progress(count, total, status=''):
    """Displays an inline progress bar"""
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [132]:
def make_soup(start_url):
    """Make a request and parse with BeautifulSoup"""
    try:
        html = requests.get(start_url).text
    except HTTPError as e:
        if e.code == 503:
            to = int(e.hdrs.get("retry-after", 30))
            print("Got 503. Retrying after {0:d} seconds.".format(to))
            time.sleep(to)
            #continue
        else:
            raise
    bs = BeautifulSoup(html, 'lxml')
    return bs

In [169]:
def get_links(start_url):
    """Fetch all the links to the next page"""
    count = 0
    links =[start_url]
    soup = make_soup(start_url)
    all_links = soup.find('div', class_="pagination" ).find_all('a')
    total = len(all_links)
    for link in all_links:
        if link:
            links.append('https://wallethub.com/'+link['href'])
        progress(count, total - 1, "Fetching links")
        count += 1
    return links

In [170]:
get_links('https://wallethub.com/d/indigo-platinum-mastercard-2294c/?r=1')

['https://wallethub.com/d/indigo-platinum-mastercard-2294c/?r=1',
 'https://wallethub.com//d/indigo-platinum-mastercard-2294c/?p=2#reviews']

In [171]:
def get_data(start_url):
    """Fetch the data from each link"""
    links = get_links(start_url)
    count = 0
    #total = len(links)
    content = []
    rating = []
    
    for link in links:
            soup = make_soup(link)
            progress(count, total - 1, "Fetching data")
            count += 1
            for review in soup.find_all('div', class_="content", itemprop="description"):
                content.append(review.text.strip('\n'))
            for rate in soup.find_all('meta', itemprop="ratingValue"):
                if rate['content'] != "3.6":
                    rating.append(rate['content'])
                
    review_dict = {'review':content}
    rating_dict = {'rating':rating}
    
    with open('review38.json', 'a') as f:
        json.dump(review_dict,f)
    with open('rating38.json', 'a') as f:
        json.dump(rating_dict,f)

In [172]:
df = get_data('https://wallethub.com/d/indigo-platinum-mastercard-2294c/?r=1')

In [173]:
df = pd.read_json('review38.json')
df.tail()

,review
47,THIS IS THE WORST CREDIT CARD I HAVE EVER HAD....
48,Horrible service with this card. It takes a pa...
49,Not a good card for rebuilding your credit
50,Applied for this card multiple times and never...
51,"This card is supposed to be for bad credit, bu..."


In [174]:
df = pd.read_json('rating38.json')
df.tail()

,rating
49,1.0
50,1.0
51,1.0
52,1.0
53,1.0
